###### Get Clip

In [1]:
!pip install --upgrade youtube_dl

     |████████████████████████████████| 1.9MB 11.5MB/s 


In [2]:
!youtube-dl https://www.youtube.com/watch?v=91_G8iaokk8
!mv *.mkv clip.mkv

[youtube] 91_G8iaokk8: Downloading webpage
[download] Destination: Master Yoda Quote (FEAR) _ Star Wars I - The Phantom Menace (1999)-91_G8iaokk8.f136.mp4
[download] 100% of 4.52MiB in 00:00
[download] Destination: Master Yoda Quote (FEAR) _ Star Wars I - The Phantom Menace (1999)-91_G8iaokk8.f251.webm
[download] 100% of 797.54KiB in 00:00
[ffmpeg] Merging formats into "Master Yoda Quote (FEAR) _ Star Wars I - The Phantom Menace (1999)-91_G8iaokk8.mkv"
Deleting original file Master Yoda Quote (FEAR) _ Star Wars I - The Phantom Menace (1999)-91_G8iaokk8.f136.mp4 (pass -k to keep)
Deleting original file Master Yoda Quote (FEAR) _ Star Wars I - The Phantom Menace (1999)-91_G8iaokk8.f251.webm (pass -k to keep)


In [3]:
# Convert

import moviepy.editor as mp

start = 0
end = 45

clip_paths = []

for i in range(start, end, 10):
  clip = mp.VideoFileClip("clip.mkv")

  end = min(clip.duration, i+10)

  clip = clip.subclip(i,end)

  clip.audio.write_audiofile("audio_" + str(i) + ".mp3")
  clip_paths.append("audio_" + str(i) + ".mp3")

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)1761280/45929032 bytes (3.8%)4005888/45929032 bytes (8.7%)6471680/45929032 bytes (14.1%)8986624/45929032 bytes (19.6%)11427840/45929032 bytes (24.9%)14000128/45929032 bytes (30.5%)16392192/45929032 bytes (35.7%)18964480/45929032 bytes (41.3%)21610496/45929032 bytes (47.1%)24428544/45929032 bytes (53.2%)26894336/45929032 bytes (58.6%)29540352/45929032 bytes (64.3%)320

100%|██████████| 221/221 [00:00<00:00, 628.22it/s]

[MoviePy] Done.


[MoviePy] Writing audio in audio_10.mp3


100%|██████████| 221/221 [00:00<00:00, 587.53it/s]

...

[MoviePy] Writing audio in audio_30.mp3


100%|██████████| 221/221 [00:00<00:00, 540.27it/s]


[MoviePy] Done.
[MoviePy] Writing audio in audio_40.mp3


100%|██████████| 203/203 [00:00<00:00, 516.19it/s]

[MoviePy] Done.


In [4]:
# Play Audio 

from IPython.display import Audio
Audio("audio_0.mp3")

###### Transcribe

In [5]:
!pip install -q transformers

     |████████████████████████████████| 2.0MB 12.0MB/s 
     |████████████████████████████████| 3.2MB 53.8MB/s 
     |████████████████████████████████| 890kB 49.6MB/s 


In [6]:
 from transformers import Wav2Vec2Tokenizer, Wav2Vec2ForCTC

 # load model and tokenizer
 tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
 model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
import librosa
import torch

cc = ""

# Doing one by one instead of batched to save RAM
for path in clip_paths:
    input_audio, _ = librosa.load(path, 
                                  sr=16000)

    input_values = tokenizer(input_audio, return_tensors="pt", padding="longest").input_values

    logits = model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = tokenizer.batch_decode(predicted_ids)[0]

    cc += transcription + "  "

print(cc)

NOW FEEL YOU COLD SIR AFRAID ARE YOU NO SIR  SAY THROUGH YOU WE CAN BE MINDFUL OF YOUR FEELING YOUR THOUGHTS DWELL ON YOUR MOTHER  I MISS HER AFRAID TO LOSE HER I THINK  WHAT HAS I GOT TO DO WITH ANYTHING EVERYTHING O  EAR IS THE PATH TO THE DARK SIGH FEAR TEAS TO ANGER ANGRILEAS TO HATE HATE LEADS TO SUFFERING  W I SENSE MUCH FAIR IN YOW  


#### Experiments (Ignore)

In [2]:
!pip install -q transformers
!pip install -q datasets

     |████████████████████████████████| 2.0MB 8.3MB/s 
     |████████████████████████████████| 3.2MB 39.5MB/s 
     |████████████████████████████████| 890kB 53.7MB/s 
     |████████████████████████████████| 194kB 9.1MB/s 
     |████████████████████████████████| 245kB 14.9MB/s 
     |████████████████████████████████| 112kB 11.6MB/s 


In [ ]:
# Calculate Hz
data = wavfile.read(file_name)
framerate = data[0]
sounddata = data[1]
time = np.arange(0,len(sounddata))/framerate
print('Sample rate:',framerate,'Hz')
print('Total time:',len(sounddata)/framerate,'s')

##### Model

In [3]:
 from transformers import Wav2Vec2Tokenizer, Wav2Vec2ForCTC
 from datasets import load_dataset
 import soundfile as sf
 import torch

 # load model and tokenizer
 tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-large-960h")
 model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h")

 # define function to read in sound file
 def map_to_array(batch):
     speech, _ = sf.read(batch["file"])
     batch["speech"] = speech
     return batch

 # load dummy dataset and read soundfiles
 ds = load_dataset("patrickvonplaten/librispeech_asr_dummy", "clean", split="validation")
 ds = ds.map(map_to_array)

 # tokenize
 input_values = tokenizer(ds["speech"][:2], return_tensors="pt", padding="longest").input_values  # Batch size 1

 # retrieve logits
 logits = model(input_values).logits

 # take argmax and decode
 predicted_ids = torch.argmax(logits, dim=-1)
 transcription = tokenizer.batch_decode(predicted_ids)

/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:358: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  FutureWarning,


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Dataset librispeech_asr downloaded and prepared to /root/.cache/huggingface/datasets/librispeech_asr/clean/2.1.0/468ec03677f46a8714ac6b5b64dba02d246a228d92cbbad7f3dc190fa039eab1. Subsequent calls will reuse this data.


In [4]:
transcription

['BECAUSE YOU ARE SLEEPING INSTEAD OF CONQUERING THE LOVELY ROSE PRINCESS HAS BECOME A FIDDLE WITHOUT A BOAW WILE POOR SHAGGY SITS THERE A COOING DOVE',
 'HE HAS GONE AND GONE FOR GOOD ANSWERED POLYCHROME WHO HAD MANAGED TO SQUEEZE INTO THE ROOM BESIDE THE DRAGON AND HAD WITNESSED THE OCCURRENCES WITH MUCH INTEREST']